In [88]:
import pandas as pd
import numpy as np
from timeit import timeit
from datetime import date, timedelta, datetime
import time

from FileLoad import Final_Load
from Skills import complex_skills

today = date.today()
tomorrow = (today + timedelta(days = 1)).strftime("%Y-%m-%d")

In [106]:
### Get data and mutate
df0 = Final_Load()

name_sort = {'Escalated':0, 'Unscheduled':1,'PNP Released':2,'Past Due':3,'Scheduled':4}
rm_sort = {'EMR Remote': 0, 'HIH - Other': 1, 'Onsite':2,'Offsite':3}
# age_sort = {21: 0, 0: 1, 20:2, 15:3, 10:4, 5:5}

df0['name_sort'] = df0['Outreach Status'].map(name_sort)
df0['rm_sort'] = df0['Retrieval Group'].map(rm_sort)
# df0['age_sort'] = df0['Group Number'].map(age_sort)

# df0['ORG_list'] = 0
df0['ORG Stat'] = 0 

df_dummy_status = pd.get_dummies(df0['Outreach Status'])
df_dummy_age = pd.get_dummies(df0['Group Number'])
df1 = df0.join(df_dummy_status).join(df_dummy_age)

### Unique Numbers count and status
df2 = df1.groupby(['PhoneNumber']).agg({'PhoneNumber':'count','Cluster':'mean','Escalated':'sum','PNP Released':'sum','Past Due':'sum','Scheduled':'sum','Unscheduled':'sum',}).rename(columns={'PhoneNumber':'OutreachID Count','Cluster':'Cluster_Avg'})
df2 = df2.reset_index()

### Add info to main line and reskill
df3 = pd.merge(df0,df2, on='PhoneNumber')
df3 = complex_skills(df3)

### Sort Order and drop Dups
df4 = df3.sort_values(by = ['rm_sort', 'name_sort', 'Unscheduled', 'Cluster_Avg'], ascending= [True, True, False, True]).reset_index(drop = True)

df4 = df4.drop_duplicates(['PhoneNumber']).reset_index(drop = True)
df4['ORG Stat'] = 1

Skipping line 57213: field larger than field limit (1000)
Skipping line 96725: field larger than field limit (1000)
Skipping line 97632: field larger than field limit (1000)
Skipping line 302863: field larger than field limit (1000)
Skipping line 303803: field larger than field limit (1000)
Skipping line 305333: field larger than field limit (1000)


In [94]:
### Concat Column for ORGs to Unique Numbers
Phone_List = df4[df4['OutreachID Count'] > 1]['PhoneNumber']
df_clean = df0[['OutreachID', 'PhoneNumber']]
df_phone = pd.DataFrame()
new_row = pd.DataFrame()

def phone(number):
    df1 = df_clean[df_clean['PhoneNumber'] == number]['OutreachID'].tolist()
    string_ORG = ""
    for i in df1:
        string_ORG = string_ORG +str(i) +'|'
    new_row = [[number, string_ORG]]
    return new_row

for i in Phone_List:
    df_phone = df_phone.append(phone(i), ignore_index=True)

In [97]:
# phone(8008604230)

In [98]:
startTime_1 = time.time()


executionTime_1 = (time.time() - startTime_1)
print("-----------------------------------------------")
print('Time: ' + str(executionTime_1))
print("-----------------------------------------------")

-----------------------------------------------
Time: 86.22116684913635
-----------------------------------------------


In [107]:
# df_phone

In [108]:
# df_phone['Cat ORGs'] = df_phone['Cat ORGs'].str.rstrip('dtype: object')#.str[3:]
# df_phone.columns = ['PhoneNumber', 'ORG List']
df_phone

,PhoneNumber,ORG List
0,9.375231e+09,22093445|23241175|23241895|23242715|23812387|2...
1,4.192273e+09,23836901|24321202|24985548|25354434|25555084|2...
2,2.709655e+09,24313824|25318568|25406491|25600728|26035571|2...
3,4.195178e+09,22299060|25263160|
4,8.647203e+09,24763773|25396363|25979483|25979836|25979849|2...
...,...,...
59348,4.053309e+09,26470629|26473861|
59349,4.056924e+09,26471092|26471887|
59350,6.418121e+09,26472470|26472550|
59351,9.128197e+09,26474226|26475347|26475351|


In [109]:
df4 = df4.merge(df_phone, how='left', on='PhoneNumber')

,OutreachID,Outreach Team,Client / Optum Category,Recommended Schedule Date,PhoneNumber,Address,City,State,Zip,Agent,...,rm_sort,ORG Stat,OutreachID Count,Cluster_Avg,Escalated,PNP Released,Past Due,Scheduled,Unscheduled,ORG List
0,25979849,Sub 15,Highmark,2021-07-02,8.647203e+09,60 MAPLE RD STE 1,BUFFALO,NY,14221.0,,...,0,1,198,0,0,0,1,7,190.0,24763773|25396363|25979483|25979836|25979849|2...
1,26404002,Sub 15,WellCare,2021-07-02,2.065205e+09,13231 SE 36TH ST STE 110,BELLEVUE,WA,98006.0,,...,0,1,60,0,0,0,9,2,49.0,20328011|24044083|24458817|24500825|24503551|2...
2,25124843,Sub 15,Anthem,2021-07-02,4.196965e+09,8423 MARKET ST STE 101,YOUNGSTOWN,OH,44512.0,,...,0,1,24,0,0,0,0,0,24.0,20591866|21229087|22676088|24401234|24884602|2...
3,25723033,15 to 59,UHC,2021-07-02,8.889527e+09,3003 N CENTRAL AVE STE 800,PHOENIX,AZ,85012.0,,...,0,1,32,0,2,0,5,1,24.0,21102038|21951737|21963633|22092948|22272333|2...
4,25626648,Sub 15,UHC,2021-07-02,8.005278e+09,3000 HOSPITAL DR,BATAVIA,OH,45103.0,,...,0,1,21,0,1,0,1,1,18.0,25363497|25440620|25626648|25698705|25731261|2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164434,26478819,15 to 59,NaN,2021-07-02,8.509266e+09,2382 CRAWFORDVILLE HWY,CRAWFORDVILLE,FL,32327.0,,...,3,1,1,0,0,0,0,1,0.0,NaN
164435,26479104,Sub 15,NaN,2021-07-02,6.103611e+09,161 WILMINGTON W CHESTER PIKE,CHADDS FORD,PA,19317.0,,...,3,1,1,0,0,0,0,1,0.0,NaN
164436,26479241,Sub 15,NaN,2021-07-02,9.287744e+09,1750 S RAILROAD SPRINGS BLVD STE 3,FLAGSTAFF,AZ,86001.0,,...,3,1,1,0,0,0,0,1,0.0,NaN
164437,26479335,15 to 59,NaN,2021-07-02,8.594986e+09,101 BRANDON WAY STE B,MOUNT STERLING,KY,40353.0,,...,3,1,1,0,0,0,0,1,0.0,NaN


In [ ]:
os.getcwd()

In [102]:
df_phone.to_csv('df_phone.csv', index=False)
